In [25]:

import os
from vidstab import VidStab
import cv2

from concurrent.futures import ThreadPoolExecutor, as_completed


def stabilize_video(input_path, output_path):
    """
    Stabilise a single video. If unhappy with the stabilisation, play with the settings of the
    VidStab class or stabilize method parameters.

    Refer to vidstab docs:
    https://pypi.org/project/vidstab/
    """
    # Create an instance of the VidStab class
    # try different kp_methods - Faster - ORB, FAST, slower MSER, SIFT, DENSE
    stabilizer = VidStab(kp_method="FAST",
                         threshold=42,
                         nonmaxSuppression=False)

    stabilizer.stabilize(input_path=input_path,
                         output_path=output_path,
                         smoothing_window = 45,
                         border_size=-75)


def stabilize_videos(input_folder, output_folder):
    """
    Stabilises videos sequentially
    """
    # Ensure output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        # Check if the file is a video (you can customize this to check for specific extensions)
        if filename.endswith(('.mov')): # could be '.avi', '.mp4', '.mkv'
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, filename)

            stabilize_video(input_path, output_path)
            print(f'Stabilized video saved as {output_path}')


In [26]:

def stabilize_videos_in_parallel(input_folder, output_folder):
    """
    Stabilize video files from a specified input folder in parallel,

    This function searches for video files in the input folder with supported file extensions
    (e.g., .mov, .mp4), and processes each video using the `stabilize_video` function in parallel
    using a thread pool executor. Stabilized output is saved in the output folder.

    """
    # Create output folder if needed
    os.makedirs(output_folder, exist_ok=True)

    # Find video files to stabilize (add other formats if needed)
    video_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.mov', '.mp4'))]

    # Prepare input and output paths
    input_paths = [os.path.join(input_folder, f) for f in video_files]
    output_paths = [os.path.join(output_folder, f) for f in video_files]

    # Stabilize videos in parallel
    with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
        future_to_video = {executor.submit(stabilize_video, input_path, output_path): video_file
                           for input_path, output_path, video_file in zip(input_paths, output_paths, video_files)}

        # Wait for the tasks to complete
        for future in as_completed(future_to_video):
            video_file = future_to_video[future]
            try:
                future.result()
                print(f'Successfully stabilized {video_file}')
            except Exception as e:
                print(f'Error stabilizing {video_file}: {e}')

In [27]:
input_folder = "original"
output_folder = "stabilised"

stabilize_videos_in_parallel(input_folder, output_folder)

Successfully stabilized MVI_E7512.MOV
Successfully stabilized MVI_E7487.MOV
Successfully stabilized MVI_E7403.MOV
